In [1]:
import opt_mo
import numpy as np
import sympy as sym
import itertools

sym.init_printing()


//anaconda3/envs/opt-mo/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
p_1, p_2, p_3, p_4 = sym.symbols("p_1, p_2, p_3, p_4")
q_1, q_2, q_3, q_4 = sym.symbols("q_1, q_2, q_3, q_4")


In [3]:
p = (p_1, p_2, p_3, p_4)
q = (q_1, q_2, q_3, q_4)


In [4]:
pi_1, pi_2, pi_3, pi_4 = sym.symbols("pi_1, pi_2, pi_3, pi_4")
pi = (pi_1, pi_2, pi_3, pi_4)


In [5]:
R, S, T, P = sym.symbols("R, S, T, P")

In [48]:
def pd_utility(S_p, x, y):
    """
    Returns the utility as a function
    of p, q vectors. Based on Press and Dyson, 2012.
    """
    B = sym.Matrix(opt_mo.make_B(S_p, x, y))
    unit = sym.Matrix(opt_mo.make_B((1, 1, 1, 1), x, y))
    return B.det() / unit.det()

**Theorem 1 Proof**

As discrubed in Section 2 the utility of a memory one player against another is given by the steady states of M
multiplied by the payoffs.

In [6]:
M = opt_mo.mem_one_match_markov_chain(player=p, opponent=q)


In [7]:
ss = opt_mo.steady_states(M, pi)


In [8]:
v = sym.Matrix([[ss[pi_1]], [ss[pi_2]], [ss[pi_3]], [ss[pi_4]]])


In [9]:
first_ss = v[0].collect(p_2 * p_4).collect(p_2 * p_3).collect(p_3 * p_4)

In [10]:
second_ss = v[1].collect(p_1 * p_3).collect(p_1 * p_4).collect(p_3 * p_4)

In [11]:
(sym.fraction(second_ss)[-1] - sym.fraction(first_ss)[-1]).simplify()

In [12]:
third_ss = v[2].factor().collect(p_1 * p_2).collect(p_1 * p_4).collect(p_2 * p_4)

In [13]:
fourth_ss = v[3].factor()

In [14]:
(sym.fraction(third_ss)[-1] - sym.fraction(fourth_ss)[-1]).expand()

In [15]:
(sym.fraction(third_ss)[-1] - sym.fraction(second_ss)[-1]).simplify()

In [16]:
utility = v.dot(np.array([R, S, T, P]))

In [46]:
S_p = sym.Matrix([[R, S, T, P]])

In [49]:
utility = pd_utility(S_p, p, q)

In [59]:
numerator, denominator = sym.fraction(utility.simplify())

In [60]:
numerator

In [61]:
numerator_elements = [[numerator.coeff(f1 * f2).factor() * f1 * f2 for f2 in p] for f1 in p]

In [62]:
flat_elements = list(itertools.chain.from_iterable(numerator_elements))

In [63]:
cross_prod = sum(flat_elements) / 2

In [64]:
cross_prod

In [66]:
sym.latex(sum(flat_elements) / 2).replace("\\left", "").replace("\\right", "")

'- p_{1} p_{2} (q_{1} - q_{3}) (P q_{2} - P - T q_{4}) + p_{1} p_{3} (q_{1} - q_{2}) (P q_{3} - S q_{4}) + p_{1} p_{4} (q_{1} - q_{4}) (S q_{2} - S - T q_{3}) + p_{2} p_{3} (q_{2} - q_{3}) (P q_{1} - P - R q_{4}) - p_{2} p_{4} (q_{3} - q_{4}) (R q_{2} - R - T q_{1} + T) + p_{3} p_{4} (q_{2} - q_{4}) (R q_{3} - S q_{1} + S)'

In [67]:
linear_expr = numerator.subs({p_2: 0, p_3: 0, p_4: 0}).coeff(p_1).factor() * p_1
linear_expr += numerator.subs({p_1: 0, p_3: 0, p_4: 0}).coeff(p_2).factor() * p_2
linear_expr += numerator.subs({p_1: 0, p_2: 0, p_4: 0}).coeff(p_3).factor() * p_3
linear_expr += numerator.subs({p_1: 0, p_2: 0, p_3: 0}).coeff(p_4).factor() * p_4

In [68]:
linear_expr

In [69]:
sym.latex(linear_expr).replace("\\left", "").replace("\\right", "")

'p_{1} q_{1} (P q_{2} - P - T q_{4}) - p_{2} (q_{3} - 1) (P q_{2} - P - T q_{4}) + p_{3} (- P q_{1} q_{2} + P q_{2} q_{3} + P q_{2} - P q_{3} + R q_{2} q_{4} - S q_{2} q_{4} + S q_{4}) + p_{4} (- R q_{2} q_{4} + R q_{4} + S q_{2} q_{4} - S q_{2} - S q_{4} + S + T q_{1} q_{4} - T q_{3} q_{4} + T q_{3} - T q_{4})'

In [70]:
constant = numerator.subs({p_2: 0, p_3: 0, p_4: 0, p_1: 0})

In [71]:
constant

In [72]:
sym.latex(constant)

'- P q_{2} + P + T q_{4}'

In [33]:
((constant - linear_expr - cross_prod) - numerator).simplify()

In [36]:
((cross_prod + linear_expr + constant) - numerator).factor()

**Denominator**

In [59]:
denominator_elements = [[denominator.coeff(f1 * f2) * f1 * f2 for f2 in p] for f1 in p]

In [60]:
flat_elements = list(itertools.chain.from_iterable(denominator_elements))

In [61]:
cross_prod = sum(flat_elements) / 2

In [62]:
cross_prod

In [63]:
sym.latex(cross_prod).replace("\\left", "").replace("\\right", "")

'p_{1} p_{2} (q_{1} q_{2} - q_{1} q_{4} - q_{1} - q_{2} q_{3} + q_{3} q_{4} + q_{3}) + p_{1} p_{3} (- q_{1} q_{3} + q_{1} q_{4} + q_{2} q_{3} - q_{2} q_{4}) + p_{1} p_{4} (- q_{1} q_{2} + q_{1} q_{3} + q_{1} + q_{2} q_{4} - q_{3} q_{4} - q_{4}) + p_{2} p_{3} (- q_{1} q_{2} + q_{1} q_{3} + q_{2} q_{4} + q_{2} - q_{3} q_{4} - q_{3}) + p_{2} p_{4} (- q_{1} q_{3} + q_{1} q_{4} + q_{2} q_{3} - q_{2} q_{4}) + p_{3} p_{4} (q_{1} q_{2} - q_{1} q_{4} - q_{2} q_{3} - q_{2} + q_{3} q_{4} + q_{4})'

In [64]:
linear_expr = denominator.subs({p_2: 0, p_3: 0, p_4: 0}).coeff(p_1) * p_1
linear_expr += denominator.subs({p_1: 0, p_3: 0, p_4: 0}).coeff(p_2) * p_2
linear_expr += denominator.subs({p_1: 0, p_2: 0, p_4: 0}).coeff(p_3) * p_3
linear_expr += denominator.subs({p_1: 0, p_2: 0, p_3: 0}).coeff(p_4) * p_4

In [65]:
sym.latex(linear_expr).replace("\\left", "").replace("\\right", "")

'p_{1} (- q_{1} q_{2} + q_{1} q_{4} + q_{1}) + p_{2} (q_{2} q_{3} - q_{2} - q_{3} q_{4} - q_{3} + q_{4} + 1) + p_{3} (q_{1} q_{2} - q_{2} q_{3} - q_{2} + q_{3} - q_{4}) + p_{4} (- q_{1} q_{4} + q_{2} + q_{3} q_{4} - q_{3} + q_{4} - 1)'

In [66]:
constant = denominator.subs({p_2: 0, p_3: 0, p_4: 0, p_1: 0})

In [67]:
constant

In [68]:
sym.latex(constant)

'q_{2} - q_{4} - 1'